In [2]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embeddings = model.encode(sentences)
print(embeddings)


[[-0.00904849  0.01913359  0.02657794 ... -0.00876412 -0.05602206
  -0.02163094]
 [-0.00785422  0.03018173  0.03129675 ... -0.01246878 -0.06240674
  -0.0059097 ]]


In [24]:
import chardet

# 檢測編碼
with open('dataset/ml-1m/raw/movies.dat', 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']

print(f"Detected encoding: {encoding}")

Detected encoding: ISO-8859-1


In [28]:
import pickle
from tqdm import tqdm
embeddings_dict = {}
with open('dataset/ml-1m/raw/movies.dat', 'rb') as file:
    lines = file.readlines()
    for line in tqdm(lines):
        line = line.decode('ISO-8859-1').strip().split('::')
        movie_id = line[0]
        movie_name = ''.join(line[1].split(' (')[:-1])
        movie_year = line[1].split(' (')[-1].strip(')')
        movie_cat = line[2].split('|')
        movie_cat_str = ', '.join(movie_cat)
        sentence = f"{movie_name} is a movie released in {movie_year}. It falls under the following categories: {movie_cat_str}."
        embeddings = model.encode(sentence)
        embeddings_dict[int(movie_id)] = embeddings

with open('dataset/ml-1m/processed/embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings_dict, f)

100%|██████████| 3883/3883 [00:12<00:00, 322.21it/s]


In [2]:
from data.movie_lens import MovieLensMovieData_from_embeddings
dataset = MovieLensMovieData_from_embeddings('dataset/ml-1m')
print(len(dataset[0]))

768


In [5]:
from modules.rqvae import RqVae
import torch
model = RqVae(
        input_dim=768,
        embed_dim=32,
        hidden_dim=32,
        codebook_size=256,
        n_layers=3
    )
model.load_state_dict(torch.load('model/best_model.pth'))
model.eval()
tensor = torch.tensor(dataset[3]).unsqueeze(0)
print(model(tensor, 0.1))

quantize in train mode
quantize in train mode
quantize in train mode
tensor(0.1147, grad_fn=<MeanBackward0>)


/tmp/ipykernel_24394/2109223039.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(dataset[3]).unsqueeze(0)
